# 📌 Passo a Passo: Criando o Catálogo no Metastore do Databricks - Silver

- 1️⃣ Criar um Database no Metastore
- 2️⃣ Registrar a Tabela Delta no Metastore
- 3️⃣ Validar se a tabela foi registrada corretamente

## 🔷 Criação Database

No Databricks, as tabelas são organizadas dentro de um Database. Vamos criar um database específico para armazenar os metadados.




In [0]:
%run "./00-Configuracao"

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS catalogo_de_dados
LOCATION 's3://mvp-brasileirao-2024/bronze/catalogo_de_dados';

Leitura das credenciais de forma segura pelo Spark 🔐


Caminhos definidos:
📂 Bronze: s3a://mvp-brasileirao-2024/bronze/ | 📂 Silver: s3a://mvp-brasileirao-2024/silver/ | 📂 Gold: s3a://mvp-brasileirao-2024/gold/


✅ Credenciais configuradas com sucesso!


✅ Conexão bem-sucedida! Arquivos encontrados na camada Bronze e Silver:


## 🔷 Preparação para os Registros

Antes de registrarmos, precisarmos Salvar os DataFrames no formato Delta dentro da pasta `catalogo_de_dados`

### 🔹 Conversão .csv → Delta

Criamos as pastas do Catalogo de Dados, porém agora precisamos converter os arquivos csv para Delta, para darmos inicio a criação dos Catálogos

In [0]:
dbutils.fs.rm("s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_todas_partidas", recurse=True)
dbutils.fs.rm("s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_classificacao", recurse=True)
dbutils.fs.rm("s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_estatistica_jogador", recurse=True)

Out[6]: False

### 🔹 Definição dos Paths

In [0]:
from pyspark.sql.functions import col

# 🔹 Caminhos dos arquivos CSV na Bronze
bronze_path_partidas = "s3://mvp-brasileirao-2024/bronze/BrasilSerieA_2024_TodasPartidas.csv"
bronze_path_classificacao = "s3://mvp-brasileirao-2024/bronze/BrasilSerieA_2024_ClassificacaoFinal.csv"
bronze_path_estatistica = "s3://mvp-brasileirao-2024/bronze/BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv"

# 🔹 Caminhos onde serão armazenados os arquivos Delta
delta_path_partidas = "s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_todas_partidas"
delta_path_classificacao = "s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_classificacao"
delta_path_estatistica = "s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_estatistica_jogador"

### 🔹 Criação dos Dataframes

In [0]:

# 🔹 Criar DataFrames lendo os CSV
df_partidas = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ";").csv(bronze_path_partidas)
df_classificacao = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ";").csv(bronze_path_classificacao)
df_estatistica = spark.read.option("header", True).option("inferSchema", True).option("delimiter", ";").csv(bronze_path_estatistica)

### 🔹 Salvando Dataframes em formato Delta

In [0]:
# Função para limpar os nomes das colunas
def limpar_nomes_colunas(df):
    for c in df.columns:
        novo_nome = c.strip().replace(" ", "_").replace(".", "").replace(",", "").replace(";", "").replace("{", "").replace("}", "").replace("(", "").replace(")", "").replace("\n", "").replace("\t", "").replace("-", "_")
        df = df.withColumnRenamed(c, novo_nome)
    return df

# Aplicar a limpeza das colunas
df_partidas = limpar_nomes_colunas(df_partidas)
df_classificacao = limpar_nomes_colunas(df_classificacao)
df_estatistica = limpar_nomes_colunas(df_estatistica)

# Salvar os DataFrames no formato Delta dentro da pasta catalogo_de_dados/
df_partidas.write.format("delta") \
    .mode("overwrite") \
    .save(delta_path_partidas)

df_classificacao.write.format("delta") \
    .mode("overwrite") \
    .save(delta_path_classificacao)

df_estatistica.write.format("delta") \
    .mode("overwrite") \
    .save(delta_path_estatistica)

### 🔹 Criando os Paths dos arquivos Delta

In [0]:
# Caminhos dos arquivos Delta na Bronze
delta_path_partidas = "s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_todas_partidas"
delta_path_classificacao = "s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_classificacao"
delta_path_estatistica = "s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_estatistica_jogador"

### 🔹 Leitura dos arquivos Delta via Spark

In [0]:
# 1️⃣ Ler os arquivos Delta da Bronze
df_partidas = spark.read.format("delta").load(delta_path_partidas)
df_classificacao = spark.read.format("delta").load(delta_path_classificacao)
df_estatistica = spark.read.format("delta").load(delta_path_estatistica)

## 🔷 Registro das tabelas Delta no Metastore

Agora, precisamos registrar a tabela de catálogo de dados no Metastore.

### 🔹 Registro das Tabelas no Metastore


In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.brasil_seriea_2024_todas_partidas;
CREATE TABLE catalogo_de_dados.brasil_seriea_2024_todas_partidas
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_todas_partidas/';

DROP TABLE IF EXISTS catalogo_de_dados.brasil_seriea_2024_classificacao;
CREATE TABLE catalogo_de_dados.brasil_seriea_2024_classificacao
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_classificacao/';

DROP TABLE IF EXISTS catalogo_de_dados.brasil_seriea_2024_estatistica_jogador;
CREATE TABLE catalogo_de_dados.brasil_seriea_2024_estatistica_jogador
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/bronze/catalogo_de_dados/brasil_seriea_2024_estatistica_jogador/';

### 🔹 Validar o Registro no Metastore

Agora, vamos verificar se a tabela foi registrada corretamente.



In [0]:
%sql
SHOW TABLES IN default;

database,tableName,isTemporary
default,brasil_seriea_2024_classificacao,false
default,brasil_seriea_2024_estatistica_jogador,false
default,brasil_seriea_2024_todas_partidas,false


# 📌 Adicionando o outro Catálogo - Catalogo Estatistico


## 🔷 BrasilSerieA_2024_ClassificacaoFinal.csv

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum

# Nome da tabela que queremos analisar (mudar para cada dataset da classificacao)
tabela_classificacao = "catalogo_de_dados.brasil_seriea_2024_classificacao"

# Ler os dados da classificacao diretamente do Metastore
df = spark.read.table(tabela_classificacao)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns]  # Contagem correta de nulos
)

In [0]:
col_descriptions = {
    "Posi��o": "Posição do clube na tabela de classificação",
    "Clube": "Nome do clube participante do campeonato",
    "Pts": "Pontos acumulados pelo clube na competição",
    "PJ": "Número de partidas jogadas pelo clube",
    "VIT": "Quantidade de vitórias conquistadas pelo clube",
    "E": "Quantidade de empates do clube",
    "DER": "Quantidade de derrotas do clube",
    "GM": "Gols marcados pelo clube ao longo da competição",
    "GC": "Gols sofridos pelo clube ao longo da competição",
    "SG": "Saldo de gols do clube (Gols Marcados - Gols Sofridos)",
}


In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # ✅ Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"

    # Buscar definição detalhada ou usar descrição genérica
    col_definition = col_descriptions.get(col_name, f"Descrição da coluna {col_name}")

    # Adicionar à lista
    catalog_data.append(Row(ID_Coluna=index,
                            Coluna=col_name, 
                            Tipo_de_Dado=col_type, 
                            Definição=col_definition,
                            Valores_Min_Max=f"{col_min} | {col_max}", 
                            Qtd_Valores_Registrados = col_count,
                            Qtd_Valores_Nulos=col_nulls,
                            Qtd_Valores_Distintos=col_distinct,
                            Fonte="Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil",
                            Link_Fonte="https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/",
                            Arquivo_Original= "classificacao_brasileirao_2024.csv",
                            Arquivo_S3 = "BrasilSerieA_2024_ClassificacaoFinal.csv"                                   
                            ))

# Criar o DataFrame do Catálogo de Dados
catalog_df = spark.createDataFrame(catalog_data)

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/catalog_classificacao"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_classificacao;

CREATE TABLE catalogo_de_dados.catalog_classificacao
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/bronze/catalogo_de_dados/catalog_classificacao';

In [0]:
%sql
select * from catalogo_de_dados.catalog_classificacao

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
9,GC,int,Gols sofridos pelo clube ao longo da competição,29 | 61,20,0,18,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
10,SG,int,Saldo de gols do clube (Gols Marcados - Gols Sofridos),-29 | 30,20,0,17,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
4,PJ,int,Número de partidas jogadas pelo clube,38 | 38,20,0,1,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
5,VIT,int,Quantidade de vitórias conquistadas pelo clube,6 | 23,20,0,15,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
2,Clube,string,Nome do clube participante do campeonato,Athletico-PR | Vasco da Gama,20,0,20,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
1,Posi��o,int,Posição do clube na tabela de classificação,1 | 20,20,0,20,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
8,GM,int,Gols marcados pelo clube ao longo da competição,29 | 61,20,0,15,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
3,Pts,int,Pontos acumulados pelo clube na competição,30 | 79,20,0,17,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
7,DER,int,Quantidade de derrotas do clube,5 | 22,20,0,12,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv
6,E,int,Quantidade de empates do clube,7 | 14,20,0,7,Tabela criada manualmente com base na classificação do Campeonato Brasileiro 2024 disponível no site da CNN Brasil,https://www.cnnbrasil.com.br/esportes/futebol/tabela-do-brasileirao/,classificacao_brasileirao_2024.csv,BrasilSerieA_2024_ClassificacaoFinal.csv


## 🔷 BrasilSerieA_2024_TodasPartidas.csv

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_todas_partidas = "catalogo_de_dados.brasil_seriea_2024_todas_partidas"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_todas_partidas)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns]  # Contagem correta de nulos
)

In [0]:
col_descriptions = {
    "Country": "País onde a partida foi realizada",
    "League": "Nome da liga ou campeonato (ex: Serie A, Premier League)",
    "Season": "Ano da temporada em que a partida ocorreu",
    "Date": "Data da partida no formato YYYY-MM-DD",
    "Time": "Horário da partida no formato timestamp",
    "Home": "Equipe mandante da partida",
    "Away": "Equipe visitante da partida",
    "HG": "Quantidade de gols marcados pela equipe mandante",
    "AG": "Quantidade de gols marcados pela equipe visitante",
    "Res": "Resultado da partida (H = Vitória do mandante, A = Vitória do visitante, D = Empate)",
    "PSCH": "Odds de vitória do time mandante fornecidas pela Pinnacle",
    "PSCD": "Odds de empate da partida fornecidas pela Pinnacle",
    "PSCA": "Odds de vitória do time visitante fornecidas pela Pinnacle",
    "MaxCH": "Maior odd de vitória do time mandante disponível no mercado de apostas",
    "MaxCD": "Maior odd de empate da partida disponível no mercado de apostas",
    "MaxCA": "Maior odd de vitória do time visitante disponível no mercado de apostas",
    "AvgCH": "Média das odds de vitória do time mandante disponíveis no mercado de apostas",
    "AvgCD": "Média das odds de empate da partida disponíveis no mercado de apostas",
    "AvgCA": "Média das odds de vitória do time visitante disponíveis no mercado de apostas",
    "BFECH": "Odds para a vitória do time da casa (Betfair Exchange)",
    "BFECD": "Odds para o empate (Betfair Exchange)",
    "BFECA": "Odds para a vitória do time visitante (Betfair Exchange)",
}


In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # ✅ Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"

    # Buscar definição detalhada ou usar descrição genérica
    col_definition = col_descriptions.get(col_name, f"Descrição da coluna {col_name}")

    # Adicionar à lista
    catalog_data.append(Row(ID_Coluna=index,
                            Coluna=col_name, 
                            Tipo_de_Dado=col_type, 
                            Definição=col_definition,
                            Valores_Min_Max=f"{col_min} | {col_max}", 
                            Qtd_Valores_Registrados = col_count,
                            Qtd_Valores_Nulos=col_nulls,
                            Qtd_Valores_Distintos=col_distinct,
                            Fonte="Kaggle",
                            Link_Fonte="https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship",
                            Arquivo_Original= "BRA.csv",
                            Arquivo_S3 = "BrasilSerieA_2024_TodasPartidas.csv"                            
                            ))

# Criar o DataFrame do Catálogo de Dados
catalog_df = spark.createDataFrame(catalog_data)

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/catalog_todas_partidas"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_todas_partidas;

CREATE TABLE catalogo_de_dados.catalog_todas_partidas
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/bronze/catalogo_de_dados/catalog_todas_partidas';

In [0]:
%sql
select * from catalogo_de_dados.catalog_todas_partidas

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
13,PSCA,double,Odds de vitória do time visitante fornecidas pela Pinnacle,1.15 | 29.58,4939,1,956,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
14,MaxCH,double,Maior odd de vitória do time mandante disponível no mercado de apostas,1.09 | 29.0,4940,0,406,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
15,MaxCD,double,Maior odd de empate da partida disponível no mercado de apostas,2.63 | 12.5,4940,0,303,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
16,MaxCA,double,Maior odd de vitória do time visitante disponível no mercado de apostas,0.0 | 36.0,4940,0,780,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
19,AvgCA,double,Média das odds de vitória do time visitante disponíveis no mercado de apostas,1.15 | 26.58,4940,0,870,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
20,BFECH,double,Odds para a vitória do time da casa (Betfair Exchange),1.24 | 10.0,235,4705,124,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
21,BFECD,double,Odds para o empate (Betfair Exchange),2.94 | 6.6,235,4705,42,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
22,BFECA,double,Odds para a vitória do time visitante (Betfair Exchange),1.39 | 20.0,235,4705,97,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
5,Time,timestamp,Horário da partida no formato timestamp,2025-04-05 00:00:00 | 2025-04-05 23:50:00,4940,0,35,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv
6,Home,string,Equipe mandante da partida,America MG | Vitoria,4940,0,36,Kaggle,https://www.kaggle.com/datasets/gabrielmeireles/brazilian-football-championship,BRA.csv,BrasilSerieA_2024_TodasPartidas.csv


## 🔷 BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, count, countDistinct, min, max, sum

# Nome da tabela que queremos analisar (mudar para cada dataset da Silver)
tabela_estatistica_jogador = "catalogo_de_dados.brasil_seriea_2024_estatistica_jogador"

# Ler os dados da Silver diretamente do Metastore
df = spark.read.table(tabela_estatistica_jogador)

# Criar DataFrame com estatísticas (incluindo contagem de nulos correta)
metadata_df = df.agg(
    *[count(col(c)).alias(f"{c}_count") for c in df.columns],  # Contagem total de registros NÃO NULOS
    *[countDistinct(col(c)).alias(f"{c}_distinct") for c in df.columns],  # Distintos
    *[min(col(c)).alias(f"{c}_min") for c in df.columns],  # Valor mínimo
    *[max(col(c)).alias(f"{c}_max") for c in df.columns],  # Valor máximo
    *[sum(col(c).isNull().cast("int")).alias(f"{c}_nulls") for c in df.columns]  # Contagem correta de nulos
)

In [0]:
col_descriptions = {
    "Jogador": "Nome do jogador que participou da partida",
    "Time": "Nome do clube do jogador",
    "#": "Número da camisa do jogador na partida",
    "Na��o": "Nacionalidade do jogador",
    "Pos": "Posição do jogador dentro de campo",
    "Idade": "Idade do jogador na data da partida (anos e dias)",
    "Min": "Minutos jogados pelo atleta na partida",
    "Gols": "Número de gols marcados na partida pelo jogador",
    "Assis": "Número de assistências realizadas pelo jogador",
    "PB": "Pênaltis cobrados pelo jogador",
    "PT": "Pênaltis tentados pelo jogador (considerando acertos e erros)",
    "TC": "Total de chutes realizados (não inclui cobranças de pênalti)",
    "CaG": "Chutes a gol (não inclui cobranças de pênalti)",
    "CrtsA": "Quantidade de cartões amarelos recebidos",
    "CrtV": "Quantidade de cartões vermelhos recebidos",
    "Contatos": "Número total de vezes que o jogador tocou na bola (conta-se receber, driblar e passar como um toque)",
    "Div": "Número de desarmes bem-sucedidos realizados pelo jogador",
    "Crts": "Número de interceptações defensivas feitas pelo jogador",
    "Bloqueios": "Número de bloqueios defensivos (interceptação de chutes, passes ou cruzamentos perigosos)",
    "xG": "Gols esperados (Expected Goals) com base na posição e situação do chute",
    "npxG": "Gols esperados (Expected Goals) excluindo pênaltis",
    "xAG": "Assistências esperadas (Expected Assists) com base na jogada",
    "SCA": "Ações que resultaram em chutes a gol (Shot-Creating Actions)",
    "GCA": "Ações que resultaram em gols (Goal-Creating Actions)",
    "Cmp": "Número de passes completados com sucesso",
    "Att": "Total de passes tentados",
    "Cmp%": "Percentual de passes completados com sucesso",
    "PrgP": "Passes progressivos (passes em direção ao gol adversário)",
    "Condu��es": "Número total de conduções de bola bem-sucedidas (domínio de bola com os pés)",
    "PrgC": "Conduções progressivas (conduções para frente que resultam em ganho de espaço)",
    "Tent": "Número total de tentativas de drible",
    "Suc": "Número de dribles bem-sucedidos",
    "Data": "Data da partida disputada",
}

In [0]:
# Criar lista de metadados para o Catálogo de Dados
catalog_data = []

# Iterar sobre as colunas da tabela Silver
for index, col_name in enumerate(df.columns, start=1):
    col_type = df.select(col(col_name)).schema.fields[0].dataType.simpleString()  # Tipo da coluna
    col_count = metadata_df.collect()[0][f"{col_name}_count"]  # Contagem total (não nulos)
    col_distinct = metadata_df.collect()[0][f"{col_name}_distinct"]  # Valores distintos
    col_nulls = metadata_df.collect()[0][f"{col_name}_nulls"]  # ✅ Contagem correta de valores nulos
    col_min = metadata_df.collect()[0][f"{col_name}_min"] if f"{col_name}_min" in metadata_df.columns else "N/A"
    col_max = metadata_df.collect()[0][f"{col_name}_max"] if f"{col_name}_max" in metadata_df.columns else "N/A"

    # Buscar definição detalhada ou usar descrição genérica
    col_definition = col_descriptions.get(col_name, f"Descrição da coluna {col_name}")

    # Adicionar à lista
    catalog_data.append(Row(ID_Coluna=index,
                            Coluna=col_name, 
                            Tipo_de_Dado=col_type, 
                            Definição=col_definition,
                            Valores_Min_Max=f"{col_min} | {col_max}", 
                            Qtd_Valores_Registrados = col_count,
                            Qtd_Valores_Nulos=col_nulls,
                            Qtd_Valores_Distintos=col_distinct,
                            Fonte="Kaggle",
                            Link_Fonte="https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024",
                            Arquivo_Original= "database.csv",
                            Arquivo_S3 = "BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv"
                        ))

# Criar o DataFrame do Catálogo de Dados
catalog_df = spark.createDataFrame(catalog_data)

In [0]:
# Caminho no S3 para salvar o Catálogo Estatístico
delta_catalogo_estatistico_path = "s3://mvp-brasileirao-2024/bronze/catalogo_de_dados/catalog_estatistica_jogador"

dbutils.fs.rm(delta_catalogo_estatistico_path, recurse=True)
print("🚨 Arquivos antigos do Delta removidos com sucesso!")

# Salvar a tabela Delta com as estatísticas
catalog_df.write.format("delta").mode("overwrite").save(delta_catalogo_estatistico_path)

print("✅ Catálogo Estatístico salvo no formato Delta com sucesso!")

🚨 Arquivos antigos do Delta removidos com sucesso!
✅ Catálogo Estatístico salvo no formato Delta com sucesso!


In [0]:
%sql
DROP TABLE IF EXISTS catalogo_de_dados.catalog_estatistica_jogador;

CREATE TABLE catalogo_de_dados.catalog_estatistica_jogador
USING DELTA
LOCATION 's3://mvp-brasileirao-2024/bronze/catalogo_de_dados/catalog_estatistica_jogador';

In [0]:
%sql
select * from catalogo_de_dados.catalog_estatistica_jogador

ID_Coluna,Coluna,Tipo_de_Dado,Definição,Valores_Min_Max,Qtd_Valores_Registrados,Qtd_Valores_Nulos,Qtd_Valores_Distintos,Fonte,Link_Fonte,Arquivo_Original,Arquivo_S3
29,Condu��es,double,Número total de conduções de bola bem-sucedidas (domínio de bola com os pés),0.0 | 106.0,11681,1,94,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
30,PrgC,double,Conduções progressivas (conduções para frente que resultam em ganho de espaço),0.0 | 17.0,11681,1,16,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
31,Tent,double,Número total de tentativas de drible,0.0 | 16.0,11681,1,17,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
32,Suc,double,Número de dribles bem-sucedidos,0.0 | 10.0,11681,1,11,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
33,Data,date,Data da partida disputada,2024-04-13 | 2024-12-08,11682,0,108,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
17,Div,double,Número de desarmes bem-sucedidos realizados pelo jogador,0.0 | 12.0,11681,1,13,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
18,Crts,int,Número de interceptações defensivas feitas pelo jogador,0 | 8,11682,0,9,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
19,Bloqueios,double,"Número de bloqueios defensivos (interceptação de chutes, passes ou cruzamentos perigosos)",0.0 | 7.0,11681,1,8,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
20,xG,double,Gols esperados (Expected Goals) com base na posição e situação do chute,0.0 | 1.7,11681,1,18,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
21,npxG,double,Gols esperados (Expected Goals) excluindo pênaltis,0.0 | 1.7,11681,1,18,Kaggle,https://www.kaggle.com/datasets/eduardopalmieri/brasileiro-player-stats-2024,database.csv,BrasilSerieA_2024_EstatisticaJogadorPorPartida.csv
